## **Step 1: load raw patient data, convert hospital name**

In [0]:
%sh
wget https://github.com/dcolinmorgan/AXA_AE_app/blob/main/coord_fulldiag_UVI_min.npy.gz?raw=tr
wget https://github.com/dcolinmorgan/AXA_AE_app/blob/main/axa_p.zip?raw=tr
unzip  /databricks/driver/axa_p.zip?raw=tr
#ls -lrth

--2022-06-21 07:03:41--  https://github.com/dcolinmorgan/AXA_AE_app/blob/main/coord_fulldiag_UVI_min.npy.gz?raw=tr
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/dcolinmorgan/AXA_AE_app/raw/main/coord_fulldiag_UVI_min.npy.gz [following]
--2022-06-21 07:03:41--  https://github.com/dcolinmorgan/AXA_AE_app/raw/main/coord_fulldiag_UVI_min.npy.gz
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dcolinmorgan/AXA_AE_app/main/coord_fulldiag_UVI_min.npy.gz [following]
--2022-06-21 07:03:41--  https://raw.githubusercontent.com/dcolinmorgan/AXA_AE_app/main/coord_fulldiag_UVI_min.npy.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.git

In [0]:
df=spark.read.parquet('file:/databricks/driver/AE_AXA_dat_full.parquet')
list_a = ['pneumonia', 'COPD','asthma','resp','lung','pulm']#,'~Cancer']
df2 = df.where(df._c10.rlike("|".join(["(" + pat + ")" for pat in list_a])))
from pyspark.sql.types import IntegerType
list_b = ['Cancer']
df2 = df2.where(~df._c10.rlike("|".join(["(" + pat + ")" for pat in list_b])))
df2 = df2.withColumn("_c4", df2["_c4"].cast(IntegerType()))

In [0]:
newColumns=['id','cd9_loc','sess','sex','age','cd9_code','mini_loc','loc1','date','tmp','diag1','diag2','tmp']
from functools import reduce
oldColumns = df.schema.names
df = reduce(lambda df, idx: df.withColumnRenamed(oldColumns[idx], newColumns[idx]), range(len(oldColumns)), df)

In [0]:
from pyspark.sql.functions import *

df = df.withColumn('cd9_loc', regexp_replace('cd9_loc', 'RH', 'Ruttonjee Hospital'))
df = df.withColumn('cd9_loc', regexp_replace('cd9_loc', 'PYN','Pamela Youde Nethersole Eastern Hospital'))
df = df.withColumn('cd9_loc', regexp_replace('cd9_loc', 'QEH','Queen Elizabeth Hospital'))
df = df.withColumn('cd9_loc', regexp_replace('cd9_loc', 'CMC','Caritas Medical Centre'))
df = df.withColumn('cd9_loc', regexp_replace('cd9_loc', 'KWH','Caritas Medical Centre'))
df = df.withColumn('cd9_loc', regexp_replace('cd9_loc', 'TMH','Tuen Mun Hospital'))
df = df.withColumn('cd9_loc', regexp_replace('cd9_loc', 'PWH','Prince of Wales Hospital'))
df = df.withColumn('cd9_loc', regexp_replace('cd9_loc', 'NDH','North District Hospital'))
df = df.withColumn('cd9_loc', regexp_replace('cd9_loc', 'YCH','Yan Chai Hospital'))
df = df.withColumn('cd9_loc', regexp_replace('cd9_loc', 'UCH','United Christian Hospital'))
df = df.withColumn('cd9_loc', regexp_replace('cd9_loc', 'QMH','Queen Mary Hospital'))
df = df.withColumn('cd9_loc', regexp_replace('cd9_loc', 'PWH','Princess Margaret Hospital'))
df = df.withColumn('cd9_loc', regexp_replace('cd9_loc', 'POH','Pok Oi Hospital'))
df = df.withColumn('cd9_loc', regexp_replace('cd9_loc', 'TKO','Tseung Kwan O Hospital'))
df = df.withColumn('cd9_loc', regexp_replace('cd9_loc', 'AHN','Alice Ho Miu Ling Nethersole Hospital'))
df = df.withColumn('cd9_loc', regexp_replace('cd9_loc', 'SJH','St. John Hospital'))
df = df.withColumn('cd9_loc', regexp_replace('cd9_loc', 'NLT','North Lantau Hospital'))
df = df.withColumn('cd9_loc', regexp_replace('cd9_loc', 'TSH','Tang Shiu Kin Hospital'))
df = df.withColumn('cd9_loc', regexp_replace('cd9_loc', 'PMH','Princess Margaret Hospital'))


df.printSchema()
df.show()

root
 |-- id: string (nullable = true)
 |-- cd9_loc: string (nullable = true)
 |-- sess: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- cd9_code: string (nullable = true)
 |-- mini_loc: string (nullable = true)
 |-- loc1: string (nullable = true)
 |-- date: string (nullable = true)
 |-- tmp: string (nullable = true)
 |-- diag1: string (nullable = true)
 |-- diag2: string (nullable = true)
 |-- tmp: string (nullable = true)

+------+--------------------+-----------+---+---+--------+--------+--------------+----------+---+--------------------+------------------+---+
|    id|             cd9_loc|       sess|sex|age|cd9_code|mini_loc|          loc1|      date|tmp|               diag1|             diag2|tmp|
+------+--------------------+-----------+---+---+--------+--------+--------------+----------+---+--------------------+------------------+---+
|714099|Caritas Medical C...|AE15124834C|  M| 80|      PA|     SKM|  SHEK KIP MEI|2015-12-17

## **Step 2: load weather data from github actions scrubber**

In [0]:
!git clone https://github.com/dcolinmorgan/aqi-stations-scraper.git

fatal: destination path 'aqi-stations-scraper' already exists and is not an empty directory.


In [0]:
import glob,os,pandas as pd,numpy as np

In [0]:
cc=pd.DataFrame()#(columns=['date','pm25','pm10','o3','no2','so2','co','loc'])
files=glob.glob('/databricks/driver/aqi-stations-scraper/data/japan-aqi/*')
for file in files:
    data=pd.read_csv(file,sep=' |,')
    data['loc']=os.path.basename(file).split(',')[0]
    cc=cc.append(data)
    
data2=cc[['date','pm25','pm10','o3','no2','so2','co','loc']]
data2.date=data2.date.astype(np.datetime64)
data2['year']=data2.date.dt.year
data2['week']=data2.date.dt.week

/databricks/python/lib/python3.9/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
<command-3933843656499856>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['year']=data2.date.dt.year
<command-3933843656499856>:11: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  data2['week']=data2.date.dt.week
<command-3933843656499856>:11: SettingWithCopyWarning: 
A value is t

In [0]:
data = spark.createDataFrame(data2)

In [0]:
from pyspark.sql.functions import to_timestamp
df.select(to_timestamp(df.date, 'yyyy-MM-dd').alias('dt')).collect()

Out[65]: [Row(dt=datetime.datetime(2015, 12, 17, 0, 0)),
 Row(dt=datetime.datetime(2015, 12, 21, 0, 0)),
 Row(dt=datetime.datetime(2015, 12, 19, 0, 0)),
 Row(dt=datetime.datetime(2015, 12, 23, 0, 0)),
 Row(dt=datetime.datetime(2015, 12, 21, 0, 0)),
 Row(dt=datetime.datetime(2015, 12, 19, 0, 0)),
 Row(dt=datetime.datetime(2015, 12, 21, 0, 0)),
 Row(dt=datetime.datetime(2015, 12, 22, 0, 0)),
 Row(dt=datetime.datetime(2015, 12, 18, 0, 0)),
 Row(dt=datetime.datetime(2015, 12, 19, 0, 0)),
 Row(dt=datetime.datetime(2015, 12, 23, 0, 0)),
 Row(dt=datetime.datetime(2015, 12, 19, 0, 0)),
 Row(dt=datetime.datetime(2015, 12, 20, 0, 0)),
 Row(dt=datetime.datetime(2015, 12, 18, 0, 0)),
 Row(dt=datetime.datetime(2015, 12, 23, 0, 0)),
 Row(dt=datetime.datetime(2015, 12, 17, 0, 0)),
 Row(dt=datetime.datetime(2015, 12, 18, 0, 0)),
 Row(dt=datetime.datetime(2015, 12, 20, 0, 0)),
 Row(dt=datetime.datetime(2015, 12, 21, 0, 0)),
 Row(dt=datetime.datetime(2015, 12, 17, 0, 0)),
 Row(dt=datetime.datetime(2015,

In [0]:
df.show()

+------+--------------------+-----------+---+---+--------+--------+--------------+----------+---+--------------------+------------------+---+
|    id|             cd9_loc|       sess|sex|age|cd9_code|mini_loc|          loc1|      date|tmp|               diag1|             diag2|tmp|
+------+--------------------+-----------+---+---+--------+--------+--------------+----------+---+--------------------+------------------+---+
|714099|Caritas Medical C...|AE15124834C|  M| 80|      PA|     SKM|  SHEK KIP MEI|2015-12-17|CMC|        COPD (496:1)|              null|CMC|
|718967|Queen Elizabeth H...|AE15178419R|  M| 78|      PA|     WTS|  WONG TAI SIN|2015-12-21|QEH|        COPD (496:1)|   Fever (780.6:5)|QEH|
|726173|   Tuen Mun Hospital|AE152083042|  M| 84|      PA|      TM|      TUEN MUN|2015-12-19|TMH|        COPD (496:1)|      COPD (496:1)|TMH|
|731013|United Christian ...|AE15172273R|  F| 78|     EP1|      LT|       LAM TIN|2015-12-23|UCH|        COPD (496:1)|              null|UCH|
|73394

## **Step 3: calculate sparse distance matrix**

In [0]:
!pip install geopy

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-1dbc69a7-5661-4526-96a3-1a7f3c672015/bin/python -m pip install --upgrade pip' command.


In [0]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="example app")
df_loc=pd.DataFrame(columns=['lat','long','name'])
for ii,i in enumerate(pd.unique(df.toPandas()['cd9_loc'])):
    a,b,c=geolocator.geocode(str(i)+", Hong Kong").point
    df_loc[ii]=[a,b,i]
df_loc=df_loc.transpose()
df_loc.columns=['lat','long','name']
df_loc=df_loc[3:]


data = data.withColumn('loc', regexp_replace('loc', 'centralnaya-str','central'))
data = data.withColumn('loc', regexp_replace('loc', 'southern','southern island'))

data = data.withColumn('loc', regexp_replace('loc', 'southern-part of chengyang district','chengyang district'))

data_loc=pd.DataFrame(columns=['lat','long','name'])
for ii,i in enumerate(pd.unique(data.toPandas()['loc'])):
  try:
    a,b,c=geolocator.geocode(str(i)+", Hong Kong").point
  except AttributeError:
    print('no location data')
  data_loc[ii]=[a,b,i]
data_loc=data_loc.transpose()
data_loc.columns=['lat','long','name']
data_loc=data_loc[3:]

no location data
no location data
no location data
no location data


In [0]:
data_loc=data_loc[~data_loc.duplicated(['lat','long'],keep='first')]
data_loc.reset_index(inplace=True)

In [0]:
data_loc=df_loc.append(data_loc)[['lat','long','name']]
data_loc.reset_index(inplace=True)
# data_loc

In [0]:
import geopy.distance
# geopy DOES use latlon configuration
data_loc['latlon'] = list(zip(data_loc['lat'], data_loc['long']))
square = pd.DataFrame(
    np.zeros((data_loc.shape[0], data_loc.shape[0])),
    index=data_loc.index, columns=data_loc.index
)

# replacing distance.vicenty with distance.distance
def get_distance(col):
    end = data_loc.loc[col.name, 'latlon']
    return data_loc['latlon'].apply(geopy.distance.distance,
                              args=(end,),
                              ellipsoid='WGS-84'
                             )

distances = square.apply(get_distance, axis=1).T

In [0]:
data_loc['src']=data_loc['name']
data_loc['dst']=data_loc['name']

In [0]:
# np.sum((distances<5)*1)
D_D=pd.DataFrame((distances<5)*1)
D_D.index=data_loc['src']
D_D.columns=data_loc['dst']

In [0]:
E_E=pd.DataFrame(D_D.stack())#.reset_index(inplace=True)
# E_E.rename=['source','target']#.reset_index(inplace=True)#.rename(columns={'level_0':'Source','level_1':'Target', 0:'Weight'})
E_E.reset_index(inplace=True)#
distance_mat=E_E[E_E[0]>0]

In [0]:
distance=distances
distance.index=data_loc['src']
distance.columns=data_loc['dst']
distance=pd.DataFrame(distance.stack())
distance.reset_index(inplace=True)

In [0]:
from pyspark import *
from pyspark.sql import *
from graphframes import *

spark = SparkSession.builder.appName('fun').getOrCreate()
vertices=df.filter("date == 2016-03-31")#.show()
edges=spark.createDataFrame(distance_mat)
# edges=edges.filter("date == 2016-03-31")#.show()

g = GraphFrame(vertices, edges)
## Take a look at the DataFrames
g.vertices.show()
g.edges.show()
## Check the number of edges of each vertex
g.degrees.show()

/databricks/spark/python/pyspark/sql/dataframe.py:150: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
+---+-------+----+---+---+--------+--------+----+----+---+-----+-----+---+
| id|cd9_loc|sess|sex|age|cd9_code|mini_loc|loc1|date|tmp|diag1|diag2|tmp|
+---+-------+----+---+---+--------+--------+----+----+---+-----+-----+---+
+---+-------+----+---+---+--------+--------+----+----+---+-----+-----+---+

+--------------------+--------------------+---+
|                 src|                 dst|  0|
+--------------------+--------------------+---+
|Caritas Medical C...|Caritas Medical C...|  1|
|Caritas Medical C...|Queen Elizabeth H...|  1|
|Caritas Medical C...|Princess Margaret...|  1|
|Caritas Medical C...|   Yan Chai Hospital|  1|
|Caritas Medical C...|          kwai-chung|  1|
|Caritas Medical C...|        sham-shui po|  1|
|Caritas Medical C...|            mong-kok|  1|
|Queen Elizabet

## **Step 3.5: select single date**

In [0]:
df.groupby('date').count().orderBy('count',ascending=[0]).show()

+----------+-----+
|      date|count|
+----------+-----+
|2020-01-03|   86|
|2015-01-19|   80|
|2015-02-15|   78|
|2018-02-08|   77|
|2020-01-02|   76|
|2016-02-11|   75|
|2015-04-20|   74|
|2015-02-12|   72|
|2015-04-18|   72|
|2016-01-28|   71|
|2015-01-29|   70|
|2015-04-16|   70|
|2016-03-03|   69|
|2016-03-30|   69|
|2020-01-30|   68|
|2020-01-04|   68|
|2016-01-27|   68|
|2015-01-20|   68|
|2016-03-02|   67|
|2019-07-16|   67|
+----------+-----+
only showing top 20 rows



In [0]:
HOSP_A=df.filter("date == '2015-01-19'")
df.filter("date == '2015-01-19'").show()

+------+--------------------+-----------+---+---+--------+--------+------------+----------+---+--------------------+------------+---+
|    id|             cd9_loc|       sess|sex|age|cd9_code|mini_loc|        loc1|      date|tmp|               diag1|       diag2|tmp|
+------+--------------------+-----------+---+---+--------+--------+------------+----------+---+--------------------+------------+---+
| 23780|North Lantau Hosp...|AE150050441|  M| 89|       P|      TC|  TUNG CHUNG|2015-01-19|NLT|        COPD (496:1)|COPD (496:1)|NLT|
| 23780|North Lantau Hosp...|AE150050441|  M| 89|       P|      TC|  TUNG CHUNG|2015-01-19|NLT|        COPD (496:1)|COPD (496:1)|NLT|
| 60190|   Tuen Mun Hospital|AE150113262|  F| 82|     EP1|      TM|    TUEN MUN|2015-01-19|TMH|        COPD (496:1)|COPD (496:1)|TMH|
| 60190|   Tuen Mun Hospital|AE150113262|  F| 82|     EP1|      TM|    TUEN MUN|2015-01-19|TMH|        COPD (496:1)|COPD (496:1)|TMH|
| 88212|Pamela Youde Neth...|AE15008222B|  M| 83|       P|    

In [0]:
weather_A=data.filter("date == '2015-01-19'")
data.filter("date == '2015-01-19'").show()

+-------------------+-----+-----+----+----+----+----+--------------------+----+----+
|               date| pm25| pm10|  o3| no2| so2|  co|                 loc|year|week|
+-------------------+-----+-----+----+----+----+----+--------------------+----+----+
|2015-01-19 00:00:00|152.0| 59.0|36.0|37.0| 9.0|null|          kwai-chung|2015|   4|
|2015-01-19 00:00:00|158.0| 62.0|49.0|26.0| 8.0|null|             sha-tin|2015|   4|
|2015-01-19 00:00:00|161.0| 69.0|74.0| 6.0| 7.0| 7.0|             tap-mun|2015|   4|
|2015-01-19 00:00:00|158.0| 62.0|29.0|25.0| 6.0|null|     central_western|2015|   4|
|2015-01-19 00:00:00|158.0| 78.0|53.0|15.0| 5.0|13.0|          kwai-chung|2015|   4|
|2015-01-19 00:00:00|133.0|102.0|12.0|19.0|32.0|13.0|northwest-water c...|2015|   4|
|2015-01-19 00:00:00|159.0| 68.0|46.0|23.0| 7.0|null|           kwun-tong|2015|   4|
|2015-01-19 00:00:00|158.0| 65.0|16.0|72.0| 5.0| 5.0|        causeway-bay|2015|   4|
|2015-01-19 00:00:00|162.0| 76.0|37.0|36.0| 9.0| 9.0|           y

In [0]:
inc_DL=HOSP_A.groupBy(['cd9_loc']).count()#
inc_DL.show()#agg(F.count("diag1").alias("count"))

+--------------------+-----+
|             cd9_loc|count|
+--------------------+-----+
|   Yan Chai Hospital|   14|
|   Tuen Mun Hospital|   10|
|North District Ho...|    4|
|Queen Elizabeth H...|    4|
|Tseung Kwan O Hos...|    2|
|North Lantau Hosp...|    2|
|Alice Ho Miu Ling...|    4|
|Caritas Medical C...|    6|
|Prince of Wales H...|    2|
|Princess Margaret...|   10|
|Pamela Youde Neth...|   12|
| Queen Mary Hospital|    8|
|  Ruttonjee Hospital|    2|
+--------------------+-----+



In [0]:
import pyspark.pandas as ps

merged = ps.merge(ps.DataFrame(weather_A), ps.DataFrame(inc_DL), left_on='loc', right_on='cd9_loc',how='outer')
merged

# df = weather_A.join(inc_DL, ["loc", ""])

,date,pm25,pm10,o3,no2,so2,co,loc,year,week,cd9_loc,count
0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,Alice Ho Miu Ling Nethersole Hospital,4.0
1,NaT,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,Caritas Medical Centre,6.0
2,NaT,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,North District Hospital,4.0
3,NaT,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,North Lantau Hospital,2.0
4,NaT,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,Pamela Youde Nethersole Eastern Hospital,12.0
5,NaT,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,Prince of Wales Hospital,2.0
6,NaT,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,Princess Margaret Hospital,10.0
7,NaT,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,Queen Elizabeth Hospital,4.0
8,NaT,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,Queen Mary Hospital,8.0
9,NaT,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,Ruttonjee Hospital,2.0


## **Step 4: merge date & location**

In [0]:
from pyspark.sql.functions import coalesce
    
merged2=merged.to_spark()
merged2=merged2.withColumn("cd9_loc",coalesce(merged2.cd9_loc,merged2.loc)) 

In [0]:
merged2=merged2.withColumnRenamed("cd9_loc","id")#.printSchema()
merged2.show()


+-------------------+-----+----+----+----+----+----+---------------+----+----+--------------------+-----+
|               date| pm25|pm10|  o3| no2| so2|  co|            loc|year|week|                  id|count|
+-------------------+-----+----+----+----+----+----+---------------+----+----+--------------------+-----+
|               null| null|null|null|null|null|null|           null|null|null|Alice Ho Miu Ling...|    4|
|               null| null|null|null|null|null|null|           null|null|null|Caritas Medical C...|    6|
|               null| null|null|null|null|null|null|           null|null|null|North District Ho...|    4|
|               null| null|null|null|null|null|null|           null|null|null|North Lantau Hosp...|    2|
|               null| null|null|null|null|null|null|           null|null|null|Pamela Youde Neth...|   12|
|               null| null|null|null|null|null|null|           null|null|null|Prince of Wales H...|    2|
|               null| null|null|null|null|null

In [0]:

edges=spark.createDataFrame(distance_mat)
# edges=edges.filter("date == 2016-03-31")#.show()

g = GraphFrame(merged2, edges)
## Take a look at the DataFrames
g.vertices.show()
g.edges.show()
## Check the number of edges of each vertex
g.degrees.show()

+-------------------+-----+----+----+----+----+----+---------------+----+----+--------------------+-----+
|               date| pm25|pm10|  o3| no2| so2|  co|            loc|year|week|                  id|count|
+-------------------+-----+----+----+----+----+----+---------------+----+----+--------------------+-----+
|               null| null|null|null|null|null|null|           null|null|null|Alice Ho Miu Ling...|    4|
|               null| null|null|null|null|null|null|           null|null|null|Caritas Medical C...|    6|
|               null| null|null|null|null|null|null|           null|null|null|North District Ho...|    4|
|               null| null|null|null|null|null|null|           null|null|null|North Lantau Hosp...|    2|
|               null| null|null|null|null|null|null|           null|null|null|Pamela Youde Neth...|   12|
|               null| null|null|null|null|null|null|           null|null|null|Prince of Wales H...|    2|
|               null| null|null|null|null|null

In [0]:
pr = g.pageRank(resetProbability=0.15, tol=0.01)
## look at the pagerank score for every vertex
pr.vertices.show()
## look at the weight of every edge
pr.edges.show()

+-------------------+-----+-----+----+----+----+----+--------------------+----+----+--------------------+-----+-------------------+
|               date| pm25| pm10|  o3| no2| so2|  co|                 loc|year|week|                  id|count|           pagerank|
+-------------------+-----+-----+----+----+----+----+--------------------+----+----+--------------------+-----+-------------------+
|2015-01-19 00:00:00|155.0| 62.0|30.0|28.0| 5.0|null|             eastern|2015|   4|             eastern| null| 1.0438444185102327|
|2015-01-19 00:00:00|146.0| 72.0|13.0|18.0|19.0|12.0|north-china insti...|2015|   4|north-china insti...| null|0.16931799864945407|
|               null| null| null|null|null|null|null|                null|null|null|   Yan Chai Hospital|   14| 0.8784219356416673|
|2015-01-19 00:00:00|145.0| 67.0|41.0|37.0|11.0| 6.0|          tung-chung|2015|   4|          tung-chung| null| 1.0682313060429922|
|               null| null| null|null|null|null|null|                null|nu